# Holy Trait Pattern 

From https://ahsmart.com/pub/holy-traits-design-patterns-and-best-practice-book/

英語なので自分用に翻訳しながら読む。

## Introduction

- Holy Traitsパターンは「Tim Holy Traits Trick (THTT)」とも呼ばれる。

### トレイトとは

- **トレイト**とは、オブジェクトの振る舞いに対応するもの。
- たとえば、
  - 鳥や蝶は飛べるので、どちらも`CanFly`トレイトをもつ
  - イルカや亀は泳げるので、`CanSwim`トレイトをもつ
  - アヒルは飛べるし泳げるので、その両方のトレイトをもつ
- 基本的にはトレイトは「そうかそうでないか」の二値的なもの
  - ただし厳密にそう決まっているわけではない。
  


### トレイトの意義

- トレイトはデータ型の使われ方についての正式な契約(formal contract）として使える。
- たとえば、
  - あるオブジェクトが`CanFly`トレイトを有していれば、そのオブジェクトに`fly`メソッドが定義されていると見込める。
  - `CanSwim`トレイトを持っていれば、`swim` 機能を呼びだせると期待できる。


Juliaではトレイトに対して組込みでサポートしているわけではないが、多重ディスパッチシステムの助けを借りれば十分にトレイトの概念を使うことができる。

以下では、Juliaでのトレイトモデルが、**holy traits**として知られるスペシャルなテクニックによっていかに実現できるのかを見ていく。

##  ユースケース：個人資産管理

- 再利用可能なソフトウェアをデザインする際、大抵の場合、抽象をデータ型として作り、それらと「振る舞い」を関連付ける。
- 「振る舞い」のモデルの１つが、型階層の利用
  - リスコフの置換原理に従い、「ある関数が呼ばれたとき、その型を派生型に置き換えられるようにする」。
  
個人資産管理のための型階層を見てみる：

<img src="https://ahsmart.com/assets/pages/design_patterns_book/abstract_type_hier4.png" alt=" Personal Asset Type Hierarchy"/>

- まず、資産(asset)の価値を決める`value`関数を定義する
- もしあらゆる資産が何かしらの金銭的な価値を持っていれば、`Asset` 階層にあるすべての型にこの関数が適用できる。
- → ほぼすべての資産は`HasValue`トレイトを示すと言える。

### 流動性

- 時折、階層の中の特定の型にのみ適用できる振る舞いというものが出てくる。
- たとえば、
  - `trade`機能は流動資産(liquid investment)でのみ有効
  - その場合、`Investment`と`Cash`には交換機能を定義し、`House`や`Apartments`には定義しないようにすることになる。

### プログラム上の取り扱い

- プログラム的には、どの資産が流動かをどう判断すればいいか？
- １つ目の方法：流動資産の型のリストに対してチェックする
  - たとえば、
  - 資産の配列があり、手っ取り早く現金に替えれるものがどれか探したいとすると、

In [6]:
abstract type Asset end
abstract type Property <: Asset end
abstract type Investment <: Asset end
abstract type Cash <: Asset end

function show_tradable_assets(assets::Vector{Asset})
    for asset in assets
        if asset isa Investment || asset isa Cash
            println("Yes, I can trade ", asset)
        else
            println("Sorry, ", asset, " is not tradable")
        end
    end
end

show_tradable_assets (generic function with 1 method)

- if文がちょっと醜い（こんな簡単な例の時点で既に）。条件が増えたらもっと悪化する。

- ２つ目の方法：Union型を使う

In [7]:
const LiquidInvestments = Union{Investment, Cash}

function show_tradable_assets(assets::Vector{Asset})
    for asset in assets
        if asset isa LiquidInvestments
            println("Yes, I can trade ", asset)
        else
            println("Sorry, ", asset, " is not tradable")
        end
    end
end

show_tradable_assets (generic function with 1 method)

### 問題点

このアプローチの課題は、

1. Union型は新しい流動資産の型が出てくる度に更新しなければならない。この種のメンテナンスは、プログラマーがその度にこのUnion型を更新することを覚えておかないといけないため、デザインの観点からよくない。
2. このUnion型は拡張に対応できない。もし他の開発者がこの「交換ライブラリ」を使っていて、新しい資産型を追加したいとしても、大本のソースコードは弄れないのでこのUnion型の定義を変えることもできない。
3. 流動的かどうかを判別する必要がある度に、このif-then-elseロジックがコードの多くの箇所で繰り返されるかもしれない。

ってなわけで、**holy traits**パターンを使って解決しましょう。

## Holy Traitsパターンの実装 

まずは一連の型定義から：

In [8]:
abstract type Asset end

abstract type Property <: Asset end
abstract type Investment <: Asset end
abstract type Cash <: Asset end

abstract type House <: Property end
abstract type Apartment <: Property end

abstract type FixedIncome <: Investment end
abstract type Equity <: Investment end

- `Asset`の下に`Property`, `Investment`, `Cash`がある
- `Property`の下に`House`と`Apartment`がある
- `Investment`の下に`FixedIncome`と`Equity`がある

具象型もいくつか作ってみましょ：

In [9]:
struct Residence <: House
   location
end

struct Stock <: Equity
    symbol
    name
end

struct TreasuryBill <: FixedIncome
    cusip
end

struct Money <: Cash
    currency
    amount
end

念のための解説：

- `Residence` は人が居住し、土地のある家
- `Stock` は株式投資で、トレーディングシンボルと会社名で同定できる
- `TreasuryBIll` は短期国債で、CUSIPと呼ばれる標準識別子をもつ
- `Money`はただの現金

ちなみに、今回のトレイトの概念を説明するのにさして重要でないため、各構造体のフィールドには型をつけていない。

## トレイト型の定義

投資物件(investment)について話をすると、「公開市場で簡単に現金化できるもの」と「現金化するのにかなり手間がかかるもの」を区別できるようにしたい。前者を「流動性がある; liquid」、後者を「流動性がない; illiquid」と呼ぶことにする。たとえば、株(stock)は流動的だが、住居はそうではない。

まず、トレイトそのものを定義することから始めよう：

In [10]:
abstract type LiquidtyStyle end
struct IsLiquid <: LiquidtyStyle end
struct IsIlliquid <: LiquidtyStyle end

**Juliaにおいて、トレイトとは普通のデータ型以外の何者でもない！** `LiquidtyStyle` の全容はただの抽象型にすぎない。派生型である`IsLiquid`と`IsIlliquid`は、フィールドをもたない具象型としてかく。

## トレイトの組み込み

次に、これらのトレイトをデータ型に割り当てていく。Juliaではトレイトをまとめて割り当てていくのに、`<:` 演算子が使える。

In [11]:
# Default behavior is illiquid
LiquidityStyle(::Type) = IsIlliquid()

# Cash is always liquid
LiquidityStyle(::Type{<:Cash}) = IsLiquid()

# Any subtype of Investment is liquid
LiquidityStyle(::Type{<:Investment}) = IsLiquid()

LiquidityStyle (generic function with 3 methods)

トレイトのネーミングに標準的な慣習はないが、パッケージ製作者はよくトレイト型の語尾に"Style"や"Trait"とつけているようだ。

上の３行のコードの意味を見ていく：

- デフォルトでは全ての型は非流動的になるよう選んだ。別にデフォルトで流動的になるようにしてもよく、実際のユースケースによって決めればよい。
- `Cash`の派生型はすべて流動的になるようにした。この書き方で、具象型の`Money`型も包含している。`::Type{<:Cash}`は`Cash`の派生型すべてを指す。
- `Investment`の派生型はすべて流動的になるようにした。つまり、`FixedIncome`や`Equity`の派生型すべてが流動的であり、具象型の例でいえば、`Stock` も流動的になる。

## トレイトの挙動(behavior)の実装

どの型が流動的かを区別できたので、これらのトレイトをもつオブジェクトを引数にとるメソッドを定義しよう。まずは実に簡単なものから：

In [12]:
# The thing is tradable if it is liquid
tradable(x::T) where {T} = tradable(LiquidityStyle(T), x)
tradable(::IsLiquid, x) = true
tradable(::IsIlliquid, x) = false

tradable (generic function with 3 methods)

Juliaでは、型は第一級オブジェクトの一員である。

`tradable(x::T) where {T}` と書くことで引数の型が`T`であることを記す。既に`LiquidtyStyle`関数を定義しているので、渡した引数が`IsLiquid`トレイトと`IsIlliquid`トレイトのどちらを示すのかを導出できる。

というわけで、最初の`tradable`メソッドは、`LiquidtyStyle(T)`の返り値と自身の引数を、別の`tradable`の2引数メソッドに渡すように定義されている。簡単な例ではあるが、ディスパッチの恩恵が見られる。

#### Note

デフォルトのトレイト関数の引数を`::Type{<:Asset}`としないのはなぜかと思うかもしれないが、そうすると`Asset`型の階層下に定義された型でしか利用できなくなってしまうので使い勝手が制限されてしまう。どっちが好ましいのかは実際の使われ方による。


さて、同じトレイトを活かしたもっと興味深い関数を見てみよう。流動的な資産は市場で簡単に取引されるのだから、その市場価格も手っ取り早く探せるようにすべきだ。株式については、株式取引所から値付けサービスを呼び出すのがいいかもしれない。現金だと、市場価格は単なる貨幣額にあたる。コードに落としてみよう：

In [13]:
# The thing has a market price if it is liquid

marketprice(x::T) where {T} = marketprice(LiquidityStyle(T), x)

marketprice(::IsLiquid, x) =
    error("Please implement pricing function for ", typeof(x))

marketprice(::IsIlliquid, x) =
    error("Price for illiquid asset $x is not available.")


marketprice (generic function with 3 methods)

コードの構造はさっきの「取引可能」関数と同じである。1引数メソッドでは引数のオブジェクトのトレイトを決定し、次の2引数メソッドでは流動設備と非流動設備で異なる挙動を実装している。ここでは、`marketprice`関数は両方の場合で`error`関数を呼び出して例外をあげることにしている。もちろんこれは実際に実現したいことではなく、`Stock`と`Money`型に対して特定の値付け関数を定義したい。というわけで、やってみよう：

In [14]:
# Sample pricing functions for Money and Stock
marketprice(x::Money) = x.amount
marketprice(x::Stock) = rand(200:250)

marketprice (generic function with 5 methods)

ここでは、`Money`については単にその貨幣額を返すようにした。これはかなり単純化しており、実際にはそこから地域通貨の額（たとえばUSドルとか）を計算するかもしれない。`Stock`については、とりあえず乱数を返すようにしてある。実際には、この関数を株式価格サービスに接続することになるだろう。

理解を深めるために、以下のようなテスト関数を書いた：

In [15]:
function trait_test_cash()
    cash = Money("USD", 100.00)
    @show tradable(cash)
    @show marketprice(cash)
end

function trait_test_stock()
    aapl = Stock("AAPL", "Apple, Inc.")
    @show tradable(aapl)
    @show marketprice(aapl)
end

function trait_test_residence()
    try
        home = Residence("Los Angeles")
        @show tradable(home) # returns false
        @show marketprice(home) # exception is raised
    catch ex
        println(ex)
    end
    return true
end

function trait_test_bond()
    try
        bill = TreasuryBill("123456789")
        @show tradable(bill)
        @show marketprice(bill) # exception is raised
    catch ex
        println(ex)
    end
    return true
end


trait_test_bond (generic function with 1 method)

In [16]:
trait_test_cash()

tradable(cash) = true
marketprice(cash) = 100.0


100.0

In [17]:
trait_test_stock()

tradable(aapl) = true
marketprice(aapl) = 211


211

In [18]:
trait_test_residence()

tradable(home) = false
ErrorException("Price for illiquid asset Residence(\"Los Angeles\") is not available.")


true

In [19]:
trait_test_bond()

tradable(bill) = true
ErrorException("Please implement pricing function forTreasuryBill")


true

*Perfect!* `tradable`関数は現金と株式の両方ともを流動的だと正しく認識しているし、住居の非流動性も認識している。現金と株式については、`marketprice`関数は期待通りに値を返してくれた。住居は流動的でないので、エラーが挙がった。最後に、短期証券（TreasuryBill)は流動的ではあるが、`marketprice`関数がまだ対応していないのでエラーが挙がっている。

## トレイトを別の型階層で使う

Holy traitパターンの最も良いところは、別の型階層に属するようなオブジェクトに対しても使えるところである。「文芸」を例にして、次のような型階層を定義してみよう：

In [20]:
abstract type Literature end

struct Book <: Literature
    name
end

ここで、`Book`に`LiquidityStyle`トレイトを付与できる：

In [21]:
# assign trait
LiquidityStyle(::Type{Book}) = IsLiquid()

# sample pricing function
marketprice(b::Book) = 10.0

marketprice (generic function with 6 methods)

こうして、他の取引可能な資産と全く同じように、本を扱えるようになった。

## よくある使い方を見てみる

Holy traitパターンはオープンソースパッケージでよく使われている。いくつか例を見てみよう。

### 1. Base IteratorSize

JuliaのBaseライブラリはかなり全面的にトレイトを使っている。その一例が`Base.IteratorSize`で、その定義は`generator.jl`に書かれてある：

In [22]:
abstract type IteratorSize end
struct SizeUnknown <: IteratorSize end
struct HasLength <: IteratorSize end
struct HasShape{N} <: IteratorSize end
struct IsInfinite <: IteratorSize end

このトレイトはこれまでに学んできたものとは少し違う。二値的でないからだ。`IteratorSize`は`SizeUnknown`, `HasLength`, `HasShape{N}`, `IsInfinite`の４つの場合がありうる。`IteratorSize`関数は次のように定義されている：

In [24]:
"""
    IteratorSize(itertype::Type) -> IteratorSize
"""
IteratorSize(x) = IteratorSize(typeof(x))
IteratorSize(::Type) = HasLength() # HasLength is the default
IteratorSize(::Type{<:AbstractArray{<:Any,N}}) where {N} = HasShape{N}()
IteratorSize(::Type{Generator{I,F}}) where {I,F} = IteratorSize(I)
IteratorSize(::Type{Any}) = SizeUnknown()


LoadError: UndefVarError: Generator not defined

かなりおもしろそうに見えるので、`IsInfinite`に着目してみよう。`Base.Iterators` には無限配列を生成する関数がいくつか定義されている。たとえば、`Iterators.repeated`関数は同じ値を永遠に生成するのに使えるし、`Iterators.take`関数はシーケンスから値をピックアップするのに使える。どう動くか見てみよう：

In [25]:
collect(Iterators.take(Iterators.repeated(1), 5))

5-element Array{Int64,1}:
 1
 1
 1
 1
 1

ソースコードを見れば分かるように、`Repeated`はイテレータの派生型であり、`IteratorSize`として`IsInfinite`が割り当てられている：

```
IteratorSize(::Type{<:Repeated}) = IsInfinite()
```

こんな感じで手早く確かめられる：

In [27]:
Base.IteratorSize(Iterators.repeated(1))

Base.IsInfinite()

思ったとおり、Infiniteでした！しかし、このトレイトはどのように利用されているのだろうか？そのためにBaseライブラリの`BitArray`を見てみよう。このオブジェクトは省スペース設計のブーリアン配列である。このコンストラクタ関数は任意の反復可能オブジェクトをとることができる：

In [28]:
BitArray([isodd(x) for x in 1:5])

5-element BitArray{1}:
 1
 0
 1
 0
 1

おそらく、このコンストラクタが実際に無限なものに対しては動作できないことを理解するのはそう難しくないだろう。それゆえ、`BitArray`コンストラクタの実装はそのことを考慮に入れておかなければならない。我々は`IteratorSize`トレイトに基づいてディスパッチできるので、`BitArray`コンストラクタは幸いそのようなイテレータがやってきたとしても例外を挙げられる。

```
BitArray(itr) = gen_bitarray(IteratorSize(itr), itr)

gen_bitarray(::IsInfinite, itr) =
    throw(ArgumentError("infinite-size iterable used in BitArray constructor"))
```

動作を見てみるために、`Repeated`イテレータで`BitArray`コンストラクタを呼び出してみよう：

In [29]:
BitArray(Iterators.repeated(1))

LoadError: ArgumentError: infinite-size iterable used in BitArray constructor

### 2. AbstractPlotting 変換トレイト

`AbstractPlotting.jl`はMakieプロットシステムの一部をなす抽象プロットライブラリである。ここではデータ変換に関連するトレイトをみてみよう：

In [30]:
abstract type ConversionTrait end

struct NoConversion <: ConversionTrait end
struct PointBased <: ConversionTrait end
struct SurfaceLike <: ConversionTrait end

# By default, there is no conversion trait for any object
conversion_trait(::Type) = NoConversion()
conversion_trait(::Type{<: XYBased}) = PointBased()
conversion_trait(::Type{<: Union{Surface, Heatmap, Image}}) = SurfaceLike()

LoadError: UndefVarError: XYBased not defined

ここでは`convert_arguments`関数に使える`ConversionTrait`を定義している。ソースコードにあるように、変換ロジックは３つの異なるシナリオへと適用される：

1. 変換されない。`NoConversion`というデフォルトのトレイト型
2. `PointBased` な変換
3. `SurfaceLike` な変換

デフォルトでは、`convert_arguments`関数は変換が必要ないときは何も触らず引数をただ返す：

In [31]:
# Do not convert anything if there is no conversion trait
convert_arguments(::NoConversion, args...) = args

convert_arguments (generic function with 1 method)

それから、様々な`convert_arguments`関数が定義される。ここでは2Dプロットの関数を見てみよう：

In [32]:
"""
    convert_arguments(P, x, y)::(Vector)

Takes vectors `x` and `y` and turns it into a vector of 2D points of
the values from `x` and `y`. `P` is the plot Type (it is optional).
"""
convert_arguments(::PointBased, x::RealVector, y::RealVector) = (Point2f0.(x, y),)


LoadError: UndefVarError: RealVector not defined

## `SimpleTraits.jl`パッケージを使う

`SimpleTraits.jl`パッケージを使うとトレイトをプログラミングするときに少し簡単になるかもしれない。SimpleTraitsを使って`LiquidityStyle`の例をやり直してみよう。まず、`IsLiquid` のトレイトを次のように定義する：

In [33]:
@traitdef IsLiquid{T}

LoadError: LoadError: UndefVarError: @traitdef not defined
in expression starting at In[33]:1

この構文は`T`が何もしていないようにみえるので少しぎこちなく見えるが、実際必要である。次に、型にこのトレイトを割り当てる：

In [34]:
@traitimpl IsLiquid{Cash}
@traitimpl IsLiquid{Investment}

LoadError: LoadError: UndefVarError: @traitimpl not defined
in expression starting at In[34]:1

それから、4つのコロンの特別な構文を使って、そのトレイトを示すオブジェクトを引数にとる関数を定義する：

In [35]:
@traitfn marketprice(x::::IsLiquid) =
    error("Please implement pricing function for ", typeof(x))

@traitfn marketprice(x::::(!IsLiquid)) =
    error("Price for illiquid asset $x is not available.")

LoadError: LoadError: UndefVarError: @traitfn not defined
in expression starting at In[35]:1

流動的な場合は`x::::IsLiquid`でアノテートされた引数をもっており、一方、非流動的な場合は`x::::(!IsLiquid)`でアノテートされている。パースがうまくいくように括弧が必要なことに注意。さて、テストしてみよう：

In [36]:
marketprice(Stock("AAPL", "Apple"))
marketprice(Residence("Los Angelses"))

LoadError: Price for illiquid asset Residence("Los Angelses") is not available.

期待通り、どちらのデフォルト実装もエラーを投げている。さて、`Stock`のための値付け関数を実装して、もう一度テストしてみる：

いい感じ！お分かりの通り、SimpleTrait.jlパッケージはトレイト作成のプロセスを簡単にしてくれる。

まとめると、トレイトを使うことでコードをより拡張可能なものにすることができる。とはいいつつ、心に留めておかないといけないのが、適切なトレイトを設計するにはいくらか手間がかかるということだ。コードを拡張したい人が誰でもそのあらかじめ定義されたトレイトの利用方法を理解できるように、ドキュメンテーションも重要になる。